# Data download and pre-processing

- Workshop: **Tutorial: High performance computing with Python and RS-DAT, EO summer school 2025**

- Date: September 3, 2025 

This is the notebook used to prepare data for the workshop. It creates a Sentinel-2 RGB mosaic image in Cloud-Optimized GeoTIFF (COG) format. The dataset will be used in the workshop to demonstrate an HPC workflow.

This notebook is only to demonstrate how the example data is prepared. It is not necessary to run it for preparing the data for the workshop. You can download the prepared data directly via from [this Zenodo repository](https://zenodo.org/records/16613694). On the [Spider](https://doc.spider.surfsara.nl/en/latest/) platform, which we will use for demonstration, the dataset should be available under the following path:

In [ ]:
data_dir = "/project/remotesensing/Data/eo-summer-school"

## Search for satellite images

In [ ]:
import geopandas
import odc.stac
import pystac_client
import rioxarray # used as an extension
import shapely

In [ ]:
# Setup search client for STAC API
api_url = 'https://earth-search.aws.element84.com/v1'
client = pystac_client.Client.open(api_url)

In [ ]:
# Collection to search
collection = 'sentinel-2-c1-l2a'  # Sentinel-2, Level 2A
# Create bbox for data retrieval, assume wgs84 coordinates
bbox = [3.31, 50.8, 7.09, 53.51]  # xmin, ymin, xmax, ymax

In [ ]:
# Visualize area of interest
box = shapely.box(*bbox)
geopandas.GeoSeries(box, crs="EPSG:4326").explore()

In [ ]:
# Setup the search
search = client.search(
    collections=[collection],
    bbox=bbox,
    datetime='2025-06-12', # search for a single date for a consistent solar intensity
    query=['eo:cloud_cover<40']
)

# Inspect how many items match the search
search.matched()

## Create RGB mosaic 

In [ ]:
# Set resolution
res = 10

# Get items from the search results
items = search.item_collection()

# Load the search results as a xarray Dataset
ds = odc.stac.load(
    items,
    groupby="solar_day", # group the images within the same day
    bands=["red", "green", "blue"],
    resolution=res, # loading resolution
    chunks={'x': 4096, 'y': 4096}, # lazy loading with Dask
    bbox=bbox,
    dtype="uint16",
)

In [ ]:
# Inspect the size of ds
print(f"Dataset size: {ds.nbytes / 1e6:.2f} MB")  # in MB

In [ ]:
# Simple preprocessing function to generate RGB raster
def rgb_img(ds, vmin=0, vmax=10_000):
    """
    Generate a RGB multi-band raster.

    Sentinel-2 L2A images are provided in Digital Numbers (DNs), clip to given values.

    Parameters
    ----------
    ds : xarray.Dataset
        Dataset that contains the "red", "green", and "blue" variables
    vmin : int
        Set this as minimum value for the dataset
    vmax : int
        Set this as maximum value for the dataset

    Returns
    -------
    xarray.DataArray:
        RGB image as multi-band raster
    """
    ds_rgb = ds[["red", "green", "blue"]].to_array()
    ds_rgb = ds_rgb.clip(max=vmax, min=vmin)
    return ds_rgb

In [ ]:
ds_rgb = rgb_img(ds.isel(time=0), vmin=0, vmax=4000)  # Select the first time step
ds_rgb

## Write data to disk

In [ ]:
# Write the RGB image as a COG
tiff_output = f"{data_dir}/sentinel2_rgb_mosaic.tif"

In [ ]:
ds_rgb.rio.to_raster(
    tiff_output,
    driver="COG",
    BLOCKSIZE=4096, # same chunks as in memory
)